In [1]:
import os
import pandas as pd
import networkx as nx

from scipy.spatial.distance import cdist

In [2]:
cur_path = os.path.os.getcwd()
print(cur_path)
data_path = os.path.abspath(os.path.join(cur_path, '../nfl_data/2025/'))
print(data_path)

d:\masters-degree\masters-project\notebooks
d:\masters-degree\masters-project\nfl_data\2025


In [3]:
games = pd.read_csv(os.path.join(data_path, 'games.csv'))
players = pd.read_csv(os.path.join(data_path, 'players.csv'))
plays = pd.read_csv(os.path.join(data_path, 'plays.csv'))
week1 = pd.read_csv(os.path.join(data_path, 'tracking_week_1.csv'))

In [4]:
plays.duplicated().any()

np.False_

In [28]:
# get all games where KC is playing
games[(games['homeTeamAbbr'] == 'KC') | (games['visitorTeamAbbr'] == 'KC')]

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
10,2022091110,2022,1,9/11/2022,16:25:00,ARI,KC,21,44
16,2022091500,2022,2,9/15/2022,20:15:00,KC,LAC,27,24
35,2022092502,2022,3,9/25/2022,13:00:00,IND,KC,20,17
62,2022100213,2022,4,10/2/2022,20:20:00,TB,KC,31,41
79,2022101000,2022,5,10/10/2022,20:15:00,KC,LV,30,29
91,2022101610,2022,6,10/16/2022,16:25:00,KC,BUF,20,24
105,2022102310,2022,7,10/23/2022,16:25:00,SF,KC,23,44
134,2022110610,2022,9,11/6/2022,20:20:00,KC,TEN,20,17


In [29]:
# get the game id for the week 1 game where KC is playing
game_id = games[((games['homeTeamAbbr'] == 'KC') | (games['visitorTeamAbbr'] == 'KC')) & (games['week'] == 1)]['gameId']
game_id = game_id.values[0]
game_id

np.int64(2022091110)

In [30]:
# get all plays for the week 1 game where KC is playing and sort by quarter and gameClock. Get the playId for the first play
kc_w1_plays = plays.loc[plays['gameId'] == game_id]
kc_w1_plays = kc_w1_plays.sort_values(['quarter', 'gameClock'], ascending=[True, False])
play_id = kc_w1_plays['playId'].values[0]
play_id

np.int64(55)

In [31]:
# get all tracking data for the first play of week 1 game where KC is playing. Select only the rows where frameType is 'SNAP'

# week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['nflId'] == 40011.0)]
# week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['displayName'].str.contains('Helaire')) & (week1['frameType'] == 'SNAP')]
week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['frameType'] == 'SNAP')].sort_values('club')

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
2090649,2022091110,55,42401.0,Markus Golden,126,SNAP,2022-09-11 20:27:06.9,44.0,ARI,left,84.680000,24.67,0.00,0.00,0.00,57.91,29.62,ball_snap
2094297,2022091110,55,53565.0,Marco Wilson,126,SNAP,2022-09-11 20:27:06.9,20.0,ARI,left,77.320000,18.12,0.22,2.06,0.03,60.49,15.13,ball_snap
2091033,2022091110,55,43376.0,Nick Vigil,126,SNAP,2022-09-11 20:27:06.9,59.0,ARI,left,81.190000,32.38,0.00,0.00,0.00,93.19,117.05,ball_snap
2091417,2022091110,55,44848.0,Budda Baker,126,SNAP,2022-09-11 20:27:06.9,3.0,ARI,left,74.780000,22.69,0.93,0.99,0.09,71.90,18.39,ball_snap
2093913,2022091110,55,53445.0,Zaven Collins,126,SNAP,2022-09-11 20:27:06.9,25.0,ARI,left,81.420000,27.29,0.00,0.00,0.01,59.57,341.55,ball_snap
2093721,2022091110,55,52539.0,Rashard Lawrence,126,SNAP,2022-09-11 20:27:06.9,90.0,ARI,left,84.370000,28.42,0.11,1.42,0.01,50.05,95.83,ball_snap
2093337,2022091110,55,52416.0,Isaiah Simmons,126,SNAP,2022-09-11 20:27:06.9,9.0,ARI,left,80.340000,24.63,3.49,3.02,0.34,54.59,354.22,ball_snap
2092377,2022091110,55,47816.0,Byron Murphy,126,SNAP,2022-09-11 20:27:06.9,7.0,ARI,left,79.160000,39.98,0.16,0.16,0.03,141.18,307.14,ball_snap
2093145,2022091110,55,49410.0,Jalen Thompson,126,SNAP,2022-09-11 20:27:06.9,34.0,ARI,left,74.820000,33.70,0.71,1.07,0.07,87.06,31.85,ball_snap
2092761,2022091110,55,47848.0,Zach Allen,126,SNAP,2022-09-11 20:27:06.9,94.0,ARI,left,84.740000,34.98,0.26,0.57,0.03,121.17,86.08,ball_snap


In [32]:
# get the height, weight, and position of all players in the first play of week 1 game where KC is playing and add it to the tracking data

first_play = week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['frameType'] == 'SNAP')].sort_values('club')

new_columns = players[['nflId', 'height', 'weight', 'position']]
# new_columns

pd.set_option('display.max_columns', None)
result = pd.merge(first_play, new_columns, on='nflId')
result

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,height,weight,position
0,2022091110,55,42401.0,Markus Golden,126,SNAP,2022-09-11 20:27:06.9,44.0,ARI,left,84.68,24.67,0.00,0.00,0.00,57.91,29.62,ball_snap,6-3,260,OLB
1,2022091110,55,53565.0,Marco Wilson,126,SNAP,2022-09-11 20:27:06.9,20.0,ARI,left,77.32,18.12,0.22,2.06,0.03,60.49,15.13,ball_snap,5-11,191,CB
2,2022091110,55,43376.0,Nick Vigil,126,SNAP,2022-09-11 20:27:06.9,59.0,ARI,left,81.19,32.38,0.00,0.00,0.00,93.19,117.05,ball_snap,6-2,235,ILB
3,2022091110,55,44848.0,Budda Baker,126,SNAP,2022-09-11 20:27:06.9,3.0,ARI,left,74.78,22.69,0.93,0.99,0.09,71.90,18.39,ball_snap,5-10,195,SS
4,2022091110,55,53445.0,Zaven Collins,126,SNAP,2022-09-11 20:27:06.9,25.0,ARI,left,81.42,27.29,0.00,0.00,0.01,59.57,341.55,ball_snap,6-4,260,OLB
5,2022091110,55,52539.0,Rashard Lawrence,126,SNAP,2022-09-11 20:27:06.9,90.0,ARI,left,84.37,28.42,0.11,1.42,0.01,50.05,95.83,ball_snap,6-2,308,NT
6,2022091110,55,52416.0,Isaiah Simmons,126,SNAP,2022-09-11 20:27:06.9,9.0,ARI,left,80.34,24.63,3.49,3.02,0.34,54.59,354.22,ball_snap,6-3,225,MLB
7,2022091110,55,47816.0,Byron Murphy,126,SNAP,2022-09-11 20:27:06.9,7.0,ARI,left,79.16,39.98,0.16,0.16,0.03,141.18,307.14,ball_snap,5-11,190,CB
8,2022091110,55,49410.0,Jalen Thompson,126,SNAP,2022-09-11 20:27:06.9,34.0,ARI,left,74.82,33.70,0.71,1.07,0.07,87.06,31.85,ball_snap,5-11,190,FS
9,2022091110,55,47848.0,Zach Allen,126,SNAP,2022-09-11 20:27:06.9,94.0,ARI,left,84.74,34.98,0.26,0.57,0.03,121.17,86.08,ball_snap,6-4,281,DE


In [33]:
players[players['displayName'] == 'Markus Golden']

,nflId,height,weight,birthDate,collegeName,position,displayName
225,42401,6-3,260,1991-03-13,Missouri,OLB,Markus Golden


In [34]:
print(week1.loc[(week1['frameType'] == 'SNAP') & (week1['displayName'] != 'football'), 's'].describe())
print()
print(week1.loc[(week1['frameType'] == 'SNAP') & (week1['displayName'] != 'football'), 'a'].describe())

count    42944.000000
mean         0.374731
std          0.783342
min          0.000000
25%          0.010000
50%          0.130000
75%          0.410000
max          8.500000
Name: s, dtype: float64

count    42944.000000
mean         0.639680
std          0.763839
min          0.000000
25%          0.030000
50%          0.380000
75%          0.980000
max         10.100000
Name: a, dtype: float64


In [35]:
G = nx.from_pandas_edgelist(result.assign(target='none'),
                            source='nflId', target='target',
                            edge_attr=True)

In [36]:
x = result.to_dict(orient='index')
for index in x:
    obj = x[index]
    nflId = obj['nflId']
    for key in obj:
        if key == 'nflId':
            continue
        G.nodes[nflId][key] = obj[key]

In [37]:
G.nodes[42401.0]

{'gameId': 2022091110,
 'playId': 55,
 'displayName': 'Markus Golden',
 'frameId': 126,
 'frameType': 'SNAP',
 'time': '2022-09-11 20:27:06.9',
 'jerseyNumber': 44.0,
 'club': 'ARI',
 'playDirection': 'left',
 'x': 84.68,
 'y': 24.67,
 's': 0.0,
 'a': 0.0,
 'dis': 0.0,
 'o': 57.91,
 'dir': 29.62,
 'event': 'ball_snap',
 'height': '6-3',
 'weight': 260,
 'position': 'OLB'}

In [38]:
G.edges

EdgeView([(42401.0, 'none'), ('none', 53565.0), ('none', 43376.0), ('none', 44848.0), ('none', 53445.0), ('none', 52539.0), ('none', 52416.0), ('none', 47816.0), ('none', 49410.0), ('none', 47848.0), ('none', 48032.0), ('none', 40011.0), ('none', 53492.0), ('none', 52440.0), ('none', 47839.0), ('none', 46243.0), ('none', 46152.0), ('none', 45695.0), ('none', 44874.0), ('none', 44822.0), ('none', 43367.0), ('none', 53655.0)])

In [39]:
# get the euclidean distance between all players in the first play of week 1 game where KC is playing
coords = result[['x', 'y']].values
dist_matrix = cdist(coords, coords, metric='euclidean')
print(dist_matrix)

[[ 0.          9.85251744  8.46310818 10.09605864  4.18234384  3.76279152
   4.34018433 16.27471966 13.37013463 10.31017459  7.83063854 10.21317776
   5.38932278  9.59550416  3.96203231 12.14214561  2.97321375  8.63891776
   4.02266081  7.47438961  3.98931072  7.11214454]
 [ 9.85251744  0.         14.77580793  5.22843189 10.04484445 12.48168658
   7.17638488 21.93730157 15.7793029  18.42053202 16.19987654 19.17186741
  14.5321196  19.26343687 11.31264779 20.57291909 12.8256033  17.54092358
   9.01035515 17.3174594  13.58311084 16.15323187]
 [ 8.46310818 14.77580793  0.         11.61827009  5.09519381  5.07877938
   7.79647356  7.86644138  6.50532859  4.40028408  3.59200501  6.35307799
   5.1758864   8.6583428  12.09266306  6.55228968  8.06102971  5.24535032
  12.46795893  9.04860763  6.51871153  5.05743018]
 [10.09605864  5.22843189 11.61827009  0.          8.07772245 11.17143679
   5.88873501 17.8361571  11.01007266 15.81915611 14.00842961 17.15776792
  13.16313033 18.11652561 13.0664

In [40]:
# create a dataframe with the euclidean distance between all players in the first play of week 1 game where KC is playing
dist_df = pd.DataFrame(dist_matrix, index=result['nflId'], columns=result['nflId'])
dist_df

nflId,42401.0,53565.0,43376.0,44848.0,53445.0,52539.0,52416.0,47816.0,49410.0,47848.0,48032.0,40011.0,53492.0,52440.0,47839.0,46243.0,46152.0,45695.0,44874.0,44822.0,43367.0,53655.0
nflId,,,,,,,,,,,,,,,,,,,,,,
42401.0,0.000000,9.852517,8.463108,10.096059,4.182344,3.762792,4.340184,16.274720,13.370135,10.310175,7.830639,10.213178,5.389323,9.595504,3.962032,12.142146,2.973214,8.638918,4.022661,7.474390,3.989311,7.112145
53565.0,9.852517,0.000000,14.775808,5.228432,10.044844,12.481687,7.176385,21.937302,15.779303,18.420532,16.199877,19.171867,14.532120,19.263437,11.312648,20.572919,12.825603,17.540924,9.010355,17.317459,13.583111,16.153232
43376.0,8.463108,14.775808,0.000000,11.618270,5.095194,5.078779,7.796474,7.866441,6.505329,4.400284,3.592005,6.353078,5.175886,8.658343,12.092663,6.552290,8.061030,5.245350,12.467959,9.048608,6.518712,5.057430
44848.0,10.096059,5.228432,11.618270,0.000000,8.077722,11.171437,5.888735,17.836157,11.010073,15.819156,14.008430,17.157768,13.163130,18.116526,13.066434,18.028802,12.737755,15.612895,11.282704,16.812367,12.813157,14.489065
53445.0,4.182344,10.044844,5.095194,8.077722,0.000000,3.159019,2.870888,12.889674,9.200440,8.376067,6.199492,9.272378,5.090305,10.040782,8.143494,10.531975,5.495089,7.669687,7.858919,8.921553,4.977730,6.442189
52539.0,3.762792,12.481687,5.078779,11.171437,3.159019,0.000000,5.532179,12.679815,10.912420,6.570426,4.100549,6.758254,2.067124,6.982585,7.052695,8.486136,3.059837,5.130156,7.734856,5.770520,1.922602,3.681358
52416.0,4.340184,7.176385,7.796474,5.888735,2.870888,5.532179,0.000000,15.395288,10.617688,11.246444,9.036067,12.057305,7.594656,12.503875,7.859548,13.396735,6.850781,10.435598,6.737136,10.982409,7.001328,9.114966
47816.0,16.274720,21.937302,7.866441,17.836157,12.889674,12.679815,15.395288,0.000000,7.633741,7.492423,9.356003,9.643656,12.006702,12.891501,19.713762,7.654241,15.386829,9.945552,20.295371,14.675762,13.667205,10.922935
49410.0,13.370135,15.779303,6.505329,11.010073,9.200440,10.912420,10.617688,7.633741,0.000000,10.002240,10.032029,12.361363,11.564865,15.049385,17.318684,11.670210,13.959388,11.573556,16.981393,15.546704,12.669069,11.558689


In [41]:
# sort the players by the euclidean distance between them

sorted_distances = dist_df.apply(lambda row: row.sort_values().values.tolist(), axis=1)
sorted_players = dist_df.apply(lambda row: row.sort_values().index.tolist(), axis=1)

print(sorted_distances)
print(sorted_players)

nflId
42401.0    [0.0, 2.9732137494636928, 3.7627915169458968, ...
53565.0    [0.0, 5.228431887287044, 7.176384883769824, 9....
43376.0    [0.0, 3.5920050111323656, 4.400284081738354, 5...
44848.0    [0.0, 5.228431887287044, 5.888735008471685, 8....
53445.0    [0.0, 2.8708883642524308, 3.159018835018244, 4...
52539.0    [0.0, 1.922602402994443, 2.0671236053995403, 3...
52416.0    [0.0, 2.8708883642524308, 4.340184327882864, 5...
47816.0    [0.0, 7.492422839108854, 7.633740891594367, 7....
49410.0    [0.0, 6.5053285850908456, 7.633740891594367, 9...
47848.0    [0.0, 2.2102036105300398, 2.442969504517003, 2...
48032.0    [0.0, 1.6620770138594627, 1.7232527382830363, ...
40011.0    [0.0, 1.6329421300217621, 2.3971024175032656, ...
53492.0    [0.0, 1.7140011668607458, 1.7244709333589823, ...
52440.0    [0.0, 2.7611591768675683, 3.2478454396722745, ...
47839.0    [0.0, 2.4080074750714604, 3.9620323067839793, ...
46243.0    [0.0, 2.2102036105300398, 2.3971024175032656, ...
46152.0    [0.0, 1

In [42]:
# get the total distance covered by the player with nflId 44874.0 in the first play of week 1 game where KC is playing

player_frames = week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['frameType'] != 'AFTER_SNAP') & (week1['frameId'] >= 55) & (week1['nflId'] == 44874.0)].sort_values('time')
pd.set_option('display.max_rows', 10)
player_frames['dis'].sum().item()
# player_frames

0.38

In [43]:
# verify if the play event 'line_set' is always at the same frameId for all players for the first play of week 1 game where KC is playing

pd.set_option('display.max_rows', 24)

# frameId of 'line_set' event for 'playId'
# week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['frameType'] != 'AFTER_SNAP') & (week1['event'] == 'line_set')].sort_values('time').iloc[0]['frameId'].item()

# check how many equals values the column have, if compare to 1 it inforns if all values are the same
week1[(week1['gameId'] == game_id) & (week1['playId'] == play_id) & (week1['frameType'] != 'AFTER_SNAP') & (week1['event'] == 'line_set')]['frameId'].nunique()

1